In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
new_df = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv")
df2 = new_df.drop(columns=['essay_id','source','prompt','fold'])
df2['generated'] = new_df['label']
df2 = df2.drop(columns=['label'])

In [ ]:
df1= pd.read_csv("/kaggle/input/daigt-one-place-all-data/concatenated.csv")
train_p = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
train_e=pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
sample = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv")

In [ ]:
df1.drop(columns=['id','prompt_id','model','kaggle_repo'],inplace = True)

In [ ]:
merged_df = pd.merge(train_e, train_p, on='prompt_id')
merged_df['text'] = merged_df['text']+' '+merged_df['source_text'] + ' ' + merged_df['instructions']
df = merged_df.drop(columns = ['id','prompt_name','instructions','source_text'])
df.drop(columns = ['prompt_id'],inplace = True)

In [ ]:
train = pd.concat([df , df1, df2], axis =0)

In [ ]:
import re
import string
import nltk
import spacy
eng = spacy.load("en_core_web_lg")
from nltk.stem import PorterStemmer
porter = PorterStemmer()
from tqdm import tqdm
tqdm.pandas()

def preprocess(text):
    doc = eng(text)
    no_stop_words = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct and not token.like_url and not token.is_digit ]
    stemmed_tokens = [porter.stem(token) for token in no_stop_words]
    return " ".join(no_stop_words)

In [ ]:
# train['final_txt'] = train['text'].progress_apply(preprocess)
test['final_txt'] = test['text'].progress_apply(preprocess)

In [ ]:
# train.to_csv("pre_train_v2.0.csv")

In [ ]:
train = pd.read_csv("/kaggle/input/llm-final-preprocess/pre_train_v2.0.csv")

In [ ]:
train.drop(columns=['Unnamed: 0'],inplace = True)

In [ ]:
train

In [ ]:
y = train['generated']

In [ ]:
import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
LOWERCASE = False
VOCAB_SIZE = 30522

In [ ]:
test

In [ ]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
dataset = Dataset.from_pandas(test[['final_txt']])
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["final_txt"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['final_txt'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['final_txt'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

In [ ]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
sub = sample

In [ ]:
clf = MultinomialNB(alpha=0.02)
#     clf2 = MultinomialNB(alpha=0.01)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
p6={'n_iter': 2500,
    'verbose': -1,
    'objective': 'cross_entropy',
    'metric': 'auc',
    'learning_rate': 0.00581909898961407, 
    'colsample_bytree': 0.78,
    'colsample_bynode': 0.8, 
    'lambda_l1': 4.562963348932286, 
    'lambda_l2': 2.97485, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898}

lgb=LGBMClassifier(**p6)
cat=CatBoostClassifier(iterations=2000,
                       verbose=0,
                       l2_leaf_reg=6.6591278779517808,
                       learning_rate=0.005599066836106983,
                       subsample = 0.4,
                       allow_const_label=True,loss_function = 'CrossEntropy')
weights = [0.1,0.3,0.3,0.3]

ensemble = VotingClassifier(estimators=[('mnb',clf),
                                        ('sgd', sgd_model),
                                        ('lgb',lgb), 
                                        ('cat', cat)
                                       ],
                            weights=weights, voting='soft', n_jobs=-1)
ensemble.fit(tf_train, y)
gc.collect()
final_preds = ensemble.predict_proba(tf_test)[:,1]
sub['generated'] = final_preds
sub.to_csv('submission.csv', index=False)
sub